You can download the in this notebook from: https://www.kaggle.com/sobhanmoosavi/us-accidents

In [ ]:
# Import the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

USacc_df = pd.read_csv('US_Accidents_June20.csv')
NJacc_df = USacc_df[USacc_df['State']=='NJ']
NJacc_df.head()

#### Extract year, month, day, weekday information from start time and endtime of accident

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Convert Start_Time and End_Time to datetypes
NJacc_df['Start_Time'] = pd.to_datetime(NJacc_df['Start_Time'], errors='coerce')
NJacc_df['End_Time'] = pd.to_datetime(NJacc_df['End_Time'], errors='coerce')

# Extract year, month, day, hour, weekday and time_duration information
NJacc_df['Year']=NJacc_df['Start_Time'].dt.year
NJacc_df['Month']=NJacc_df['Start_Time'].dt.strftime('%b')
NJacc_df['Day']=NJacc_df['Start_Time'].dt.day
NJacc_df['Hour']=NJacc_df['Start_Time'].dt.hour
NJacc_df['Weekday']=NJacc_df['Start_Time'].dt.strftime('%a')

# Extract the amount of time in the unit of minutes for each accident, round to the nearest integer
total_duration='Time_Duration(min)'
NJacc_df[total_duration]=round((NJacc_df['End_Time']-NJacc_df['Start_Time'])/np.timedelta64(1,'m'))

# Check the dataframe
NJacc_df.head()

### Severity of accident in NJ?

In [ ]:
NJacc_df.Severity.value_counts().sort_values(ascending=False)

In [ ]:
def calc_autopct(pct):
    return ('%1.0f%%' % pct) if pct > 2 else ''

plt.pie(NJacc_df.Severity.value_counts(), labels=NJacc_df.Severity.value_counts().index.tolist(),autopct=calc_autopct)
plt.title('NJ accident severity')
plt.show()

Accident severity is mostly 2 and 3

### Daytime versus Nighttime accidents

In [ ]:
NJ_curr= NJacc_df['Sunrise_Sunset'].value_counts(normalize=True).round(2)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Daytime versus Nighttime accidents')
plt.show()

most of the accidents occur in the day

### Weekday versus weekend accidents

In [ ]:
weekdays = [ 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
NJacc_df.groupby('Weekday').count()['ID'].reindex(weekdays).plot(kind='bar')
plt.title('Daytime versus Nighttime accidents')
plt.xlabel('')
plt.ylim(0, 13000)

In [ ]:
NJ_curr=NJacc_df.groupby('Weekday').count()['ID'].reindex(weekdays)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Weekday versus weekend accidents')
plt.show()

Most of the accidents occur during weekdays

### Time of most accidents

In [ ]:
NJ_curr=NJacc_df.groupby('Hour').count()['ID'].reindex(np.arange(24)).plot(linestyle='dashed',color='r')
plt.xlabel('Hours')
plt.ylim(0, 7500)
plt.title('Hours of accidents')
plt.xticks(np.arange(0, 24, step=2))
plt.show()

Most of the accidents happen during 6-8 and 16-18 i.e., office travel time.


### Accidents by county

Let's first take a look of the NJ Map

<img src="new-jersey-county-map.gif" alt="Drawing" style="width: 300px;"/>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 10))
sns.scatterplot(x='Start_Lng', y='Start_Lat', data=NJacc_df, hue='County')
plt.xlabel('Longitude')
plt.ylabel('Latitude)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 10))
sns.scatterplot(x='Start_Lng', y='Start_Lat', data=NJacc_df, hue='Severity')
plt.xlabel('Longitude')
plt.ylabel('Latitude)')
plt.show()

As we can see along some path there is higher chances of sever accidents

In [ ]:
NJ_curr= NJacc_df['County'].value_counts(normalize=True).round(2)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Accidents countywise')
plt.show()

### Accidents by city

In [ ]:
NJ_curr= NJacc_df['City'].value_counts(normalize=True).round(8)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Accidents citywise')
plt.show()

### Streetside of the accidents

In [ ]:
NJ_curr= NJacc_df['Side'].value_counts(normalize=True).round(2)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Accidents Side')
plt.show()

 Most of the accidents happen on relatively right side of the street

In [ ]:
NJ_curr= NJacc_df['Weather_Condition'].value_counts(normalize=True).round(4)
labels = [n if v > 2/100 else '' for n, v in zip(NJ_curr.index, NJ_curr)] 
plt.pie(NJ_curr, labels = labels,autopct=calc_autopct)
plt.title('Accidents Side')
plt.show()

Most of the accident happened on a clear weather day, maybe because most of the day's weather is clear.

## Predict the accident severity with different supervised machine learning algorithms

#### Drop rows with negative time

In [ ]:
# Drop the rows with td<0
negtime_outliers=NJacc_df[total_duration]<=0

# Set outliers to NAN
NJacc_df[negtime_outliers] = np.nan

# Drop rows with negative td
NJacc_df.dropna(subset=[total_duration],axis=0,inplace=True)

#### Replace outliers with median values. 

In [ ]:
n=3

median = NJacc_df[total_duration].median()
std = NJacc_df[total_duration].std()
outliers = (NJacc_df[total_duration] - median).abs() > std*n

# Set outliers to NAN
NJacc_df[outliers] = np.nan

# Fill NAN with median
NJacc_df[total_duration].fillna(median, inplace=True)

In [ ]:
feature_lst=['Source','TMC','Severity','Start_Lng','Start_Lat','Distance(mi)','Side','City','County','State','Timezone','Temperature(F)','Humidity(%)','Pressure(in)', 'Visibility(mi)', 'Wind_Direction','Weather_Condition','Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop','Sunrise_Sunset','Hour','Weekday', 'Time_Duration(min)']

In [ ]:
NJacc_feature_df=NJacc_df[feature_lst]
NJacc_feature_df.dropna(subset=NJacc_feature_df.columns[NJacc_feature_df.isnull().mean()!=0], how='any', axis=0, inplace=True)
NJacc_feature_df.shape

#### Create dummy variable

what is dummay variable?

In [ ]:
s = pd.Series(list('abca'))
s

In [ ]:
pd.get_dummies(s)

In [ ]:
NJacc_dummy = pd.get_dummies(NJacc_feature_df,drop_first=True)

#### Train and test split

In [ ]:
from sklearn.model_selection import train_test_split

# Set the target for the prediction
target='Severity'

# set X and y
y = NJacc_dummy[target]
X = NJacc_dummy.drop(target, axis=1)

# Split the data set into training and testing data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [ ]:
y_test.value_counts()

##### Using KNN algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Create a k-NN classifier with 5 neighbors
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the classifier to the data
knn.fit(X_train,y_train)

# Predict the labels for the training data X
y_pred = knn.predict(X_test)

# Get the accuracy score
accuracy=accuracy_score(y_test, y_pred)


print('KNN accuracy_score: {:.3f}.'.format(accuracy))

In [ ]:
from sklearn.metrics import recall_score
recall=recall_score(y_test, y_pred, average='weighted')
print('KNN recall_score: {:.3f}.'.format(recall))

Precision attempts to answer the following question:


$$
\text{precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}, \text{ out of all points predicted to be class } 1, \text{ what fraction were actually class } 1.
$$


$$
\text{recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}, \text{ out of all the actual data points in class } 1 \text{, what fraction did the algorithm correctly predict?}
$$

In [ ]:
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average='weighted')
print('KNN precision_score: {:.3f}.'.format(precision))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
cf_matrix=multilabel_confusion_matrix(y_test, y_pred)
print('KNN cf_matrix: ', cf_matrix)

##### Using Decision tree algorithm

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate dt_entropy, set 'entropy' as the information criterion
decisiontree = DecisionTreeClassifier(max_depth=8, criterion='entropy', random_state=1)


# Fit dt_entropy to the training set
decisiontree.fit(X_train, y_train)

# Use dt_entropy to predict test set labels
y_pred= decisiontree.predict(X_test)

# Evaluate accuracy_entropy
accuracy = accuracy_score(y_test, y_pred)


# Print accuracy_entropy
print('Decision Tree accuracy_score: {:.3f}.'.format(accuracy))

In [ ]:
from sklearn.metrics import recall_score
recall=recall_score(y_test, y_pred, average='weighted')
print('Decision tree recall_score: {:.3f}.'.format(recall))

In [ ]:
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average='weighted')
print('Decision tree precision_score: {:.3f}.'.format(precision))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
cf_matrix=multilabel_confusion_matrix(y_test, y_pred)
print('Decision tree cf_matrix: ', cf_matrix)

##### Using Random forest algorithm

In [ ]:
# Random Forest algorithm
from sklearn.ensemble import RandomForestClassifier


clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)


# Get the accuracy score
accuracy=accuracy_score(y_test, y_pred)


# Model Accuracy, how often is the classifier correct?
print("Randon forest algorithm accuracy_score: {:.3f}.".format(accuracy))

In [ ]:
from sklearn.metrics import recall_score
recall=recall_score(y_test, y_pred, average='weighted')
print('Random forest recall_score: {:.3f}.'.format(recall))

In [ ]:
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average='weighted')
print('Random forest precision_score: {:.3f}.'.format(precision))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
cf_matrix=multilabel_confusion_matrix(y_test, y_pred)
print('Random forest cf_matrix: ', cf_matrix)